<a href="https://colab.research.google.com/github/RedDawe/Reinforcement_Chess/blob/master/chessown.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install tensorflow-gpu

In [0]:
import tensorflow as tf
print(tf.__version__)
assert tf.executing_eagerly()
import numpy as np

1.15.0
False


In [0]:
  e = 0

  n = 1
  q = 2
  r = 3
  b = 4
  k = 5
  p = 6

  N = -1
  Q = -2
  R = -3
  B = -4
  K = -5
  P = -6

In [0]:
def init_board():
  board = [[r, k, b, q, n, b, k, r],
          [p, p, p, p, p, p, p, p],
          [e, e, e, e, e, e, e, e],
          [e, e, e, e, e, e, e, e],
          [e, e, e, e, e, e, e, e],
          [e, e, e, e, e, e, e, e],
          [P, P, P, P, P, P, P, P],
          [R, K, B, Q, N, B, K, R]
  ]

  board = np.array(board)

  return board

[[ 3  5  4  2  1  4  5  3]
 [ 6  6  6  6  6  6  6  6]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0]
 [-6 -6 -6 -6 -6 -6 -6 -6]
 [-3 -5 -4 -2 -1 -4 -5 -3]]
5


In [0]:
def weighted_loss(logits, labels, weights):
  return tf.keras.backend.mean(tf.keras.backend.square(logits - labels) * weights)

inputs = tf.keras.Input(shape=(8, 8, 1))
weights = tf.keras.Input(shape(8, 8, 1))

#model = tf.keras.layers.BatchNormalization()(inputs)
model = tf.keras.backend.one_hot)(inputs+p, 2*p+1)

for i in range(16):
  cell = tf.keras.layers.Conv2D(filters=8, kernel_size=[1, 1], activation='tanh', padding='same')(model)
  cell = tf.keras.layers.Conv2D(filters=4, kernel_size=[3, 3], activation='tanh', padding='same')(cell)
  cell = tf.keras.layers.BatchNormalization()(cell)
  model = tf.keras.layers.Concatenate()(list([model, cell]))

model = tf.keras.layers.Conv2D(filters=2*p, kernel_size=[1, 1], activation='softmax', padding='same')(model)

#model = tf.keras.Model(inputs=inputs, outputs=model)
model = tf.keras.Model(inputs=[inputs, weights], outputs=model)


#from functools import partial
#final_loss = partial(weighted_loss, weights=weights)


model.compile(optimizer='adam', loss='weighted_loss')
#model.compile(optimizer='adam', loss='mean_squared_error')
#model.compile(optimizer='adam', loss='categorical_crossentropy')

In [0]:
def make_move(board, side, randomness):
  if side:
    board = np.flip(board, axis=0) * -1

  if np.random.randn(1)*0.5+0.5 > randomness:
    logits =  model.predict(board)
    piece, to = decide_move(board, logits)
  else:
    piece, to = random_move(board)

  if piece and to:
    board[to[0], to[1]] = board[piece[0], piece[1]]
    board[piece[0], piece[1]] = e

    weights_arr = np.zeros([8, 8])
    weights_arr[to[0], to[1]] = 1
    weights_arr[piece[0], piece[1]] = 1

    return board, weights_arr
  else:
    return 'game_over', np.zeros([8, 8])


In [0]:
def decide_move(board, logits):
  biggest_diff = 1 #0
  piece = []
  to = []

  for i in range(8):
    for j in range(8):
      #if abs(board[i, j] - logits[i, j]) > biggest_diff:
      #if -(logits[i, j] - board[i, j]) > biggest_diff:
      if board[i, j] > 0:
        if logits[i, j] / board[i, j] < biggest diff:
          moves = get_possible_moves(board, [i, j])

          for move in moves:
            copy_board = np.copy(board)
            copy_board[move[0], move[1]] = board[i, j]
            copy_board[i, j] = e

            if not is_check(copy_board, n)
              piece = [i, j]
              to = [move[0], move[1]]

return piece, to

In [0]:
def decide_move(board, logits):
  biggest_diff = 0
  piece = []
  to = []

  for i in range(8):
    for j in range(8):
      if board[i, j] > 0:
        if 1 - logits[i, j, board[i, j]+p] > biggest_diff:
          moves = get_possible_moves(board, [i, j])

          smallest_diff = 0
          for move in moves:
            if logits[move[0], move[1], board[i, j]+p] > smallest_diff:
              copy_board = np.copy(board)
              copy_board[move[0], move[1]] = board[i, j]
              copy_board[i, j] = e

              if not is_check(copy_board, n)
                piece = [i, j]
                to = [move[0], move[1]]

return piece, to

In [0]:
def random_move(board):
  I = list(range(8))
  J = list(range(8))
  np.random.shuffle(I)
  np.random.shuffle(J)

  for i in I:
    for j in J:
      if board[i, j] > 0:
        moves = get_possible_moves(board, [i, j])

        np.random.shuffle(moves)

        for move in moves:
          copy_board = np.copy(board)
          copy_board[move[0], move[1]] = board[i, j]
          copy_board[i, j] = e

          if not is_check(copy_board, n)
            piece = [i, j]
            to = [move[0], move[1]]
            break

        if piece and to:
          break

    if piece and to:
      break

In [0]:
def is_check(board, piece):
  for i in range(8):
    for j in range(8):
      if board[i, j] == piece:
        coordinates = [i, j]

  check = False
  for i in range(8):
    for j in range(8):
      if board[i, j] > 0 and not check:
        moves = get_possible_moves(board, [i, j])
        check = coordinates in moves

  return check

In [0]:
def get_possible_moves(board, coordinates): #put together all moves that 'is_legal', are on the board and aren't taking your own pieces
  legal_moves = []
  for i in range(8):
    for j in range(8):
      if (board[coordinates[0], coordinates[1]] > 0 and board[i, j] <= 0) or (board[coordinates[0], coordinates[1]] < 0 and board[i, j] >= 0)
        if is_legal(board, coordinates, [i, j])
          legal_moves.append([i, j])

  return legal_moves

In [0]:
def is_legal(board, piece, move): #check if move is in a direction that the piece moves andif squares between start and finish are empty
  if board[piece[0], piece[1]] == n: #KINGS
    return abs(piece[0] - move[0]) <= 1 and abs(piece[1] - move[1]) <= 1:

  elif board[piece[0], piece[1]] == q: #QUEEN
    if abs(piece[0] - move[0]) == 0:
      if abs(piece[1] - move[1]) >= 2:
        return np.all(board[min([piece[1], move[1]])+1 : max([piece[1], move[1]])] == e)
      else:
        return True
    elif abs(piece[1] - move[1]) == 0:
      if abs(piece[0] - move[0]) >= 2:
        return np.all(board[min([piece[0], move[0]])+1 : max([piece[0], move[0]])] == e)
      else:
        return True
    elif abs(piece[0] - move[0]) == abs(piece[1] - move[1])
      empty = True
      for i in range(8):
        for j in range(8):
          if board[i, j] != e:
            empty = False
      return empty
    else:
      return False

  elif board[piece[0], piece[1]] == r: #ROOK
    if abs(piece[0] - move[0]) == 0:
      if abs(piece[1] - move[1]) >= 2:
        return np.all(board[min([piece[1], move[1]])+1 : max([piece[1], move[1]])] == e)
      else:
        return True
    elif abs(piece[1] - move[1]) == 0:
      if abs(piece[0] - move[0]) >= 2:
        return np.all(board[min([piece[0], move[0]])+1 : max([piece[0], move[0]])] == e)
      else:
        return True
    else:
      return False

  elif board[piece[0], piece[1]] == b: #BISHOP
    if abs(piece[0] - move[0]) == abs(piece[1] - move[1])
      empty = True
      for i in range(8):
        for j in range(8):
          if board[i, j] != e:
            empty = False
      return empty
    else:
      return False

  elif board[piece[0], piece[1]] == k: #KNIGHT
    return (abs(piece[0] - move[0]) == 2 and abs(piece[0] - move[0]) == 1) or (abs(piece[0] - move[0]) == 1 and abs(piece[0] - move[0]) == 2)

  elif board[piece[0], piece[1]] == p: #PAWN
    return abs(piece[0] - move[0]) == 1 and abs(piece[0] - move[0]) == 0

  else:
    print('u fucked up')

In [0]:
n_steps = 2000
games_to_play = 100
max_depth = 100
epochs = 1000

In [0]:
checkpoint_path = "training_0/cp.ckpt"

cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

In [0]:
for step in range(n_steps):
  replay_memory = []

  for game in range(games_to_play):
    game_memory = []
    board = init_board()

    for depth in range(max_depth):
      move, weights_arr = make_move(np.copy(board), True, step/n_steps)
      if move == 'game_over':
        win = 'black'
        break
      else:
        game_memory.append(board, move, weights_arr)

      move, weights_arr = make_move(np.copy(board), False, step/n_steps)
      if move == 'game_over':
        win = 'white'
        break
      else:
        game_memory.append(board, move, weights_arr)

    else:
      win = 'white' if np.sum(board) > 0 else 'black'

    replay_memory.append([win, game_memory]) 


  x = []
  w = []
  y = []

  for game in replay_memory:
    result, moves = game

    for board, move, weights_arr in moves:
      x.append(board)
      weights.append(weights_arr)
      if result == 'white':
        y.append(moves)
      else:
        y.append(board)
  
  model.fit(x=[x, w], y=y, epochs = epochs, callbacks=[cp_callback])

In [0]:
#model.load_weights(checkpoint_path)